<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [12]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("ESP_1.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,_3,_482,_501,_544,nac,units,indexed_at,provider_code,series_code,series_name,series_num,_496,_480
0,1995-Q1,1995-01-01,112760,quarterly,30680,1.3 GDP mp Demand (Current prices),72,b1gq,pcorr,y,0,euros,2024-05-01T02:52:37.326Z,INE-SPAIN,CNTR4757,National Total. Seasonally and calendar adjust...,Master,,
1,1995-Q2,1995-04-01,114401,quarterly,30680,1.3 GDP mp Demand (Current prices),72,b1gq,pcorr,y,0,euros,2024-05-01T02:52:37.326Z,INE-SPAIN,CNTR4757,National Total. Seasonally and calendar adjust...,Master,,
2,1995-Q3,1995-07-01,115628,quarterly,30680,1.3 GDP mp Demand (Current prices),72,b1gq,pcorr,y,0,euros,2024-05-01T02:52:37.326Z,INE-SPAIN,CNTR4757,National Total. Seasonally and calendar adjust...,Master,,
3,1995-Q4,1995-10-01,117799,quarterly,30680,1.3 GDP mp Demand (Current prices),72,b1gq,pcorr,y,0,euros,2024-05-01T02:52:37.326Z,INE-SPAIN,CNTR4757,National Total. Seasonally and calendar adjust...,Master,,
4,1996-Q1,1996-01-01,119460,quarterly,30680,1.3 GDP mp Demand (Current prices),72,b1gq,pcorr,y,0,euros,2024-05-01T02:52:37.326Z,INE-SPAIN,CNTR4757,National Total. Seasonally and calendar adjust...,Master,,


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [13]:
python_df = pd.read_pickle("ESP_1.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,...,nac,units,Type of data,GOODS AND SERVICES TRANSACTIONS,ACCOUNTING BALANCES,GFCG GOODS AND SERVICES TRANSACTION,Prices_ESA,Correction of effects,National Total,Units
0,quarterly,INE-SPAIN,30680,1.3 GDP mp Demand (Current prices),CNTR4757,National Total. Seasonally and calendar adjust...,1995-Q1,1995-01-01,112760.0,112760.0,...,00,euros,Base data,NaN,Gross domestic product at market prices,NaN,Current prices,Seasonally and calendar adjusted data,National Total,Euros
1,quarterly,INE-SPAIN,30680,1.3 GDP mp Demand (Current prices),CNTR4757,National Total. Seasonally and calendar adjust...,1995-Q2,1995-04-01,114401.0,114401.0,...,00,euros,Base data,NaN,Gross domestic product at market prices,NaN,Current prices,Seasonally and calendar adjusted data,National Total,Euros
2,quarterly,INE-SPAIN,30680,1.3 GDP mp Demand (Current prices),CNTR4757,National Total. Seasonally and calendar adjust...,1995-Q3,1995-07-01,115628.0,115628.0,...,00,euros,Base data,NaN,Gross domestic product at market prices,NaN,Current prices,Seasonally and calendar adjusted data,National Total,Euros
3,quarterly,INE-SPAIN,30680,1.3 GDP mp Demand (Current prices),CNTR4757,National Total. Seasonally and calendar adjust...,1995-Q4,1995-10-01,117799.0,117799.0,...,00,euros,Base data,NaN,Gross domestic product at market prices,NaN,Current prices,Seasonally and calendar adjusted data,National Total,Euros
4,quarterly,INE-SPAIN,30680,1.3 GDP mp Demand (Current prices),CNTR4757,National Total. Seasonally and calendar adjust...,1996-Q1,1996-01-01,119460.0,119460.0,...,00,euros,Base data,NaN,Gross domestic product at market prices,NaN,Current prices,Seasonally and calendar adjusted data,National Total,Euros


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [14]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [15]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [16]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [17]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True